In [ ]:
#### This notebook is the main version of the project ####

from fileinput import close
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import folium
from itertools import combinations
from IPython.display import HTML

#### Make Data ####

#open city files
city_names_text = open('../OIE project/Data/USCA312_labels.txt', 'r')
city_distances_text = open('../OIE project/Data/USCA312_distances.txt', 'r')


#put lines of city_names_Text in a list
city_names_list = [line.strip() for line in city_names_text]
#cut out the filler text at the beginning of the txt file
city_names_list = city_names_list[15:]

#put lines of city_names_Text in a list
city_dist_list = [line.strip() for line in city_distances_text]
#cut out the filler text at the beginning of the txt file
city_dist_list = city_dist_list[7:]
#split the individual numbers into their own elements, but they're split into sublists by line
city_dist_list = [line.split() for line in city_dist_list]
#merge all the sublists
city_dist_list = [element #return element, append to list
                    for sublist in city_dist_list #for each sublist
                        for element in sublist] #for each element in each sublist
#split the main list back up into chunks for each city
city_dist_list = [city_dist_list[city_index*len(city_names_list)
                                 :
                                 (city_index+1)*len(city_names_list)] #return a range that is a block of 312 cities
                        for city_index in range(len(city_names_list))] #for each city in city_names_list

#Create the dataframe
city_dist_df = pd.DataFrame(city_dist_list, #data is the distances list
                            index=city_names_list, #row and column indices are the city list
                            columns=city_names_list)

#Gurobi tutorial works with combinations instead so mild conversion
def distance(city1, city2):
    return int(city_dist_df[city1][city2])

#create a dict of all possible edges between cities
city_dist_dict = {(c1, c2): distance(c1, c2) for c1, c2 in combinations(city_names_list, 2)}

#swap city_dist_dict for chosen set of cities later

In [ ]:
#### Model Creation ###

## Coefficients and data ##

#Max distance for a car per edge
mpg = 9
gas_tank_size = 200
#Coach bus drivers can only drive 10 hours per day legally
#Rule of thumb in the industry is that 600 miles is the cap for a legal drive, see source below
# https://mcatransportation.com/legal-driving-hours/#:~:text=The%20basic%20rule%20of%20thumb,also%20make%20the%20trip%20safe.
#Driving range can be altered for future testing
driver_range = 600

#touring list is the required cities
touring_route_list = ['Los Angeles, CA', 'New York, NY', 'Dallas, TX', 'Toronto, ON']
#create the dict of touring cities
touring_route_dict = {(c1, c2): distance(c1, c2) for c1, c2 in combinations(touring_route_list, 2)}

#Eliminate all items where the distance is above the max range
#This step implicitly implies a constraint to not drive further than allowed for 

#put the items to be deleted in a list
# bad_keys = []
# for key in city_dist_dict:
#     if city_dist_dict[key] > driver_range:
#         bad_keys.append(key)
        

# #delete keys in list from dict
# for key in bad_keys:
#     del city_dist_dict[key]

In [ ]:
## Base model ##

m = gp.Model()

# Variables: binary variables for every possible edge
vars = m.addVars(city_dist_dict.keys(), obj=city_dist_dict, vtype=GRB.BINARY, name='x')

# Symmetric direction: Copy the object
for i, j in vars.keys():
    vars[j, i] = vars[i, j]  # edge in opposite direction

In [ ]:
## Constraints ## 

# two edges incident to each city in the touring list
# the sum of both variables must be two, meaning they are both active
mandatroy_cities = m.addConstrs(vars.sum(c, '*') == 2 for c in city_names_list)

# all other cities must have either 2 or 0 variables activated
# for x1 and x2 being binary, x1-x2 will equal 0 if they are either both on or both off
#other_cities = m.addConstrs(vars.sum(c, '*') == 2 for c in city_names_list)


#Lazy subtour constraints
def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys()
                             if vals[i, j] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < len(city_names_list):
            # add subtour elimination constr. for every pair of cities in subtour
            model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2))
                         <= len(tour)-1)

# Given a tuplelist of edges, find the shortest subtour
def subtour(edges):
    unvisited = city_names_list[:]
    cycle = city_names_list[:] # Dummy - guaranteed to be replaced
    while unvisited:  # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(thiscycle) <= len(cycle):
            cycle = thiscycle # New shortest subtour
    return cycle

In [ ]:
#### Solving the model ####

#solve
m._vars = vars
m.Params.lazyConstraints = 1
m.optimize(subtourelim)

#retrieve solution
vals = m.getAttr('x', vars)
selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)

#check if it worked 
tour = subtour(selected)
assert len(tour) <= len(city_names_list)

In [ ]:
#### Generate Map ####

#data for all the lats and longs of the cities
city_coords_df = pd.read_csv('../OIE project/Data/USCA312_coords.csv')

map = folium.Map(location=[40,-95], zoom_start = 4)

points = []
for city in tour:
  city_index = city_names_list.index(city)   
  points.append( #Append the lat and long as a tuple
    (city_coords_df.iloc[city_index]['Latitude'],
    city_coords_df.iloc[city_index]['Longitude'])
  )
points.append(points[0])
#print(tour)
#print(points)

folium.PolyLine(points).add_to(map)

map